In [ ]:
!pip install --disable-pip-version-check -q sagemaker==2.35.0

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [ ]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
import time
import json
sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
print(sess, bucket, role, region)

<sagemaker.session.Session object at 0x7f9164469dd0> sagemaker-us-east-1-774375974900 arn:aws:iam::774375974900:role/c21581a406814l826944t1w7743-SageMakerExecutionRole-V1WS98ZC9FBO us-east-1


In [ ]:
!aws s3 sync 's3://sagemaker-us-east-1-774375974900' ./

download: s3://sagemaker-us-east-1-774375974900/C1_W3_Assignment_Learner.ipynb to ./C1_W3_Assignment_Learner.ipynb
download: s3://sagemaker-us-east-1-774375974900/autopilot/automl-dm-1625687301/data-processor-models/automl-dm-1625687301-dpp1-1-e7a81b369ae74776979973e2a1b491831bd/output/model.tar.gz to autopilot/automl-dm-1625687301/data-processor-models/automl-dm-1625687301-dpp1-1-e7a81b369ae74776979973e2a1b491831bd/output/model.tar.gz
download: s3://sagemaker-us-east-1-774375974900/autopilot/automl-dm-1625687301/documentation/explainability/config/analysis_config.json to autopilot/automl-dm-1625687301/documentation/explainability/config/analysis_config.json
download: s3://sagemaker-us-east-1-774375974900/autopilot/automl-dm-1625687301/documentation/explainability/output/automl-dm-1625687301mBS92wfaVVk8-001-bb8b757a/report.ipynb to autopilot/automl-dm-1625687301/documentation/explainability/output/automl-dm-1625687301mBS92wfaVVk8-001-bb8b757a/report.ipynb
download: s3://sagemaker-us-ea

In [ ]:
!aws s3 cp 's3://dlai-practical-data-science/data/balanced/womens_clothing_ecommerce_reviews_balanced.csv' ./

download: s3://dlai-practical-data-science/data/balanced/womens_clothing_ecommerce_reviews_balanced.csv to ./womens_clothing_ecommerce_reviews_balanced.csv


In [ ]:
path = './womens_clothing_ecommerce_reviews_balanced.csv'
df = pd.read_csv(path, delimiter=',')
df.head()

,sentiment,review_body,product_category
0,-1,This suit did nothing for me. the top has zero...,Swim
1,-1,Like other reviewers i saw this dress on the ...,Dresses
2,-1,I wish i had read the reviews before purchasin...,Knits
3,-1,I ordered these pants in my usual size (xl) an...,Legwear
4,-1,I noticed this top on one of the sales associa...,Knits


In [ ]:
path_autopilot = './womens_clothing_ecommerce_reviews_balanced_for_autopilot.csv'
df[['sentiment', 'review_body']].to_csv(path_autopilot, 
                                        sep=',', 
                                        index=False)

In [ ]:
autopilot_train_s3_uri = sess.upload_data(bucket=bucket, key_prefix='autopilot/data', path=path_autopilot)
autopilot_train_s3_uri

's3://sagemaker-us-east-1-774375974900/autopilot/data/womens_clothing_ecommerce_reviews_balanced_for_autopilot.csv'

In [ ]:
!aws s3 ls $autopilot_train_s3_uri

2021-07-07 19:45:57    2253749 womens_clothing_ecommerce_reviews_balanced_for_autopilot.csv


In [ ]:
model_output_s3_uri = 's3://{}/autopilot'.format(bucket)
print(model_output_s3_uri)

s3://sagemaker-us-east-1-774375974900/autopilot


In [ ]:
timestamp = int(time.time())
auto_ml_job_name = 'automl-dm-{}'.format(timestamp)

In [ ]:
auto_ml_job_name

'automl-dm-1625687301'

In [ ]:
max_candidates = 3
automl = sagemaker.automl.automl.AutoML(
    target_attribute_name='sentiment', 
    base_job_name=auto_ml_job_name, 
    output_path=model_output_s3_uri,
    max_candidates=max_candidates,
    sagemaker_session=sess,
    role=role,
    max_runtime_per_training_job_in_seconds=1200,
    total_job_runtime_in_seconds=7200
)

In [ ]:
automl.fit(
    autopilot_train_s3_uri, 
    job_name=auto_ml_job_name, 
    wait=False, 
    logs=False
)

In [ ]:
job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name)

In [ ]:
'''use two response elements: AutoMLJobStatus and AutoMLJobSecondaryStatus,
which correspond to the primary (Completed | InProgress | Failed | Stopped | Stopping) and 
secondary (AnalyzingData | FeatureEngineering | ModelTuning etc.) job states respectively.'''

In [ ]:
while 'AutoMLJobStatus' not in job_description_response.keys() and 'AutoMLJobSecondaryStatus' not in job_description_response.keys():
    job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name)
    print('[INFO] Autopilot job has not yet started. Please wait. ')
    print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))
    print('[INFO] Waiting for Autopilot job to start...')
    sleep(15)
print('[OK] AutoML job started.')

[OK] AutoML job started.


In [ ]:
from IPython.core.display import display, HTML
display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/">processing jobs</a></b>'.format(region)))


In [ ]:
%%time
job_status = job_description_response['AutoMLJobStatus']
job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('Starting', 'AnalyzingData'):
        job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name)
        job_status = job_description_response['AutoMLJobStatus']
        job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        time.sleep(15)
    print('[OK] Data analysis phase completed.\n')    
print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))

InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress FeatureEngineering
[OK] Data analysis phase completed.

{
    "AutoMLJobArn": "arn:aws:sagemaker:us-east-1:774375974900:automl-job/automl-dm-1625687301",
    "AutoMLJobArtifacts": {
        "CandidateDefinitionNotebookLocation": "s3://sagemaker-us-east-1-774375974900/autopilot/automl-dm-1625687301/sagemaker-automl-candidates/automl-dm-1625687301-pr-1-71dca513d01140df943444531031120633cda/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb",
        "DataExplorationNotebookLo

In [ ]:
job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name)
while 'AutoMLJobArtifacts' not in job_description_response.keys(): 
    job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name)
    print('[INFO] Autopilot job has not yet generated the artifacts. Please wait. ')
    print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))
    print('[INFO] Waiting for AutoMLJobArtifacts...')
    time.sleep(15)
print('[OK] AutoMLJobArtifacts generated.')

[OK] AutoMLJobArtifacts generated.


In [ ]:
job_description_response['AutoMLJobArtifacts']

{'CandidateDefinitionNotebookLocation': 's3://sagemaker-us-east-1-774375974900/autopilot/automl-dm-1625687301/sagemaker-automl-candidates/automl-dm-1625687301-pr-1-71dca513d01140df943444531031120633cda/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb',
 'DataExplorationNotebookLocation': 's3://sagemaker-us-east-1-774375974900/autopilot/automl-dm-1625687301/sagemaker-automl-candidates/automl-dm-1625687301-pr-1-71dca513d01140df943444531031120633cda/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb'}

In [ ]:
job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name) 
while 'DataExplorationNotebookLocation' not in job_description_response['AutoMLJobArtifacts'].keys(): 
    job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name) 
    print('[INFO] Autopilot job has not yet generated the notebooks. Please wait. ')
    print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))
    print('[INFO] Waiting for DataExplorationNotebookLocation...')
    time.sleep(15)
print('[OK] DataExplorationNotebookLocation found.')   

[OK] DataExplorationNotebookLocation found.


In [ ]:
from IPython.core.display import display, HTML
generated_resources = job_description_response['AutoMLJobArtifacts']['DataExplorationNotebookLocation']
download_path = generated_resources.rsplit('/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb')[0]
job_id = download_path.rsplit('/', 1)[-1]
if not job_id: 
    print('No AutoMLJobArtifacts found.')
else: 
    display(HTML('<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/autopilot/{}/sagemaker-automl-candidates/{}/">generated notebooks</a> in S3 bucket</b>'.format(bucket, auto_ml_job_name, job_id)))

In [ ]:
%%time
job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name)
job_status = job_description_response['AutoMLJobStatus']
job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('FeatureEngineering'): 
        job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name)
        job_status = job_description_response['AutoMLJobStatus']
        job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        time.sleep(5)
    print('[OK] Feature engineering phase completed.\n')   
print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))

Completed
Completed
[OK] Feature engineering phase completed.

{
    "AutoMLJobArn": "arn:aws:sagemaker:us-east-1:774375974900:automl-job/automl-dm-1625687301",
    "AutoMLJobArtifacts": {
        "CandidateDefinitionNotebookLocation": "s3://sagemaker-us-east-1-774375974900/autopilot/automl-dm-1625687301/sagemaker-automl-candidates/automl-dm-1625687301-pr-1-71dca513d01140df943444531031120633cda/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb",
        "DataExplorationNotebookLocation": "s3://sagemaker-us-east-1-774375974900/autopilot/automl-dm-1625687301/sagemaker-automl-candidates/automl-dm-1625687301-pr-1-71dca513d01140df943444531031120633cda/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb"
    },
    "AutoMLJobConfig": {
        "CompletionCriteria": {
            "MaxAutoMLJobRuntimeInSeconds": 7200,
            "MaxCandidates": 3,
            "MaxRuntimePerTrainingJobInSeconds": 1200
        },
        "SecurityConfig": {
            "EnableInterContainerTr

In [ ]:
from IPython.core.display import display, HTML
display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/hyper-tuning-jobs/">hyper-parameter tuning jobs</a></b>'.format(region)))


In [ ]:
%%time
job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name)
job_status = job_description_response['AutoMLJobStatus']
job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('ModelTuning'):
        job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name)
        job_status = job_description_response['AutoMLJobStatus']
        job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        time.sleep(5)
    print('[OK] Model tuning phase completed.\n')   
print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))

InProgress
GeneratingExplainabilityReport
[OK] Model tuning phase completed.

{
    "AutoMLJobArn": "arn:aws:sagemaker:us-east-1:774375974900:automl-job/automl-dm-1625687301",
    "AutoMLJobArtifacts": {
        "CandidateDefinitionNotebookLocation": "s3://sagemaker-us-east-1-774375974900/autopilot/automl-dm-1625687301/sagemaker-automl-candidates/automl-dm-1625687301-pr-1-71dca513d01140df943444531031120633cda/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb",
        "DataExplorationNotebookLocation": "s3://sagemaker-us-east-1-774375974900/autopilot/automl-dm-1625687301/sagemaker-automl-candidates/automl-dm-1625687301-pr-1-71dca513d01140df943444531031120633cda/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb"
    },
    "AutoMLJobConfig": {
        "CompletionCriteria": {
            "MaxAutoMLJobRuntimeInSeconds": 7200,
            "MaxCandidates": 3,
            "MaxRuntimePerTrainingJobInSeconds": 1200
        },
        "SecurityConfig": {
            "EnableI

In [ ]:
%%time
from pprint import pprint
job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name)
pprint(job_description_response)
job_status = job_description_response['AutoMLJobStatus']
job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
print('Job status:  {}'.format(job_status))
print('Secondary job status:  {}'.format(job_sec_status))
if job_status not in ('Stopped', 'Failed'):
    while job_status not in ('Completed'):
        job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name)
        job_status = job_description_response['AutoMLJobStatus']
        job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
        print('Job status:  {}'.format(job_status))
        print('Secondary job status:  {}'.format(job_sec_status))        
        time.sleep(10)
    print('[OK] Autopilot job completed.\n')
else:
    print('Job status: {}'.format(job_status))
    print('Secondary job status: {}'.format(job_status))

{'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:774375974900:automl-job/automl-dm-1625687301',
 'AutoMLJobArtifacts': {'CandidateDefinitionNotebookLocation': 's3://sagemaker-us-east-1-774375974900/autopilot/automl-dm-1625687301/sagemaker-automl-candidates/automl-dm-1625687301-pr-1-71dca513d01140df943444531031120633cda/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb',
                        'DataExplorationNotebookLocation': 's3://sagemaker-us-east-1-774375974900/autopilot/automl-dm-1625687301/sagemaker-automl-candidates/automl-dm-1625687301-pr-1-71dca513d01140df943444531031120633cda/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb'},
 'AutoMLJobConfig': {'CompletionCriteria': {'MaxAutoMLJobRuntimeInSeconds': 7200,
                                            'MaxCandidates': 3,
                                            'MaxRuntimePerTrainingJobInSeconds': 1200},
                     'SecurityConfig': {'EnableInterContainerTrafficEncryption': False}},
 'AutoMLJobN

In [ ]:
candidates = automl.list_candidates(
    job_name=auto_ml_job_name
    sort_by='FinalObjectiveMetricValue'
)

In [ ]:
while candidates == []:
    candidates = automl.list_candidates(job_name=auto_ml_job_name)
    print('[INFO] Autopilot job is generating the candidates. Please wait.')
    time.sleep(10)
print('[OK] Candidates generated.') 

[OK] Candidates generated.


In [ ]:
print(candidates[0].keys())

dict_keys(['CandidateName', 'FinalAutoMLJobObjectiveMetric', 'ObjectiveStatus', 'CandidateSteps', 'CandidateStatus', 'InferenceContainers', 'CreationTime', 'EndTime', 'LastModifiedTime'])


In [ ]:
print(candidates[0].values())

dict_values(['automl-dm-1625687301mBS92wfaVVk8-003-1ac43cb1', {'MetricName': 'validation:accuracy', 'Value': 0.4006099998950958}, 'Succeeded', [{'CandidateStepType': 'AWS::SageMaker::ProcessingJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:774375974900:processing-job/automl-dm-1625687301-db-1-171ed83849dc417aa1eb46a7a22c91183f227', 'CandidateStepName': 'automl-dm-1625687301-db-1-171ed83849dc417aa1eb46a7a22c91183f227'}, {'CandidateStepType': 'AWS::SageMaker::TrainingJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:774375974900:training-job/automl-dm-1625687301-dpp1-1-e7a81b369ae74776979973e2a1b491831bd', 'CandidateStepName': 'automl-dm-1625687301-dpp1-1-e7a81b369ae74776979973e2a1b491831bd'}, {'CandidateStepType': 'AWS::SageMaker::TransformJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:774375974900:transform-job/automl-dm-1625687301-dpp1-csv-1-992aee68b2cc412681330948cb5505f', 'CandidateStepName': 'automl-dm-1625687301-dpp1-csv-1-992aee68b2cc412681330948cb5505f'}, {

In [ ]:
while 'CandidateName' not in candidates[0]:
    candidates = automl.list_candidates(job_name=auto_ml_job_name)
    print('[INFO] Autopilot job is generating CandidateName. Please wait. ')
    sleep(10)

print('[OK] CandidateName generated.')

[OK] CandidateName generated.


In [ ]:
while 'FinalAutoMLJobObjectiveMetric' not in candidates[0]:
    candidates = automl.list_candidates(job_name=auto_ml_job_name)
    print('[INFO] Autopilot job is generating FinalAutoMLJobObjectiveMetric. Please wait. ')
    sleep(10)
print('[OK] FinalAutoMLJobObjectiveMetric generated.')

[OK] FinalAutoMLJobObjectiveMetric generated.


In [ ]:
print(json.dumps(candidates, indent=4, sort_keys=True, default=str))

[
    {
        "CandidateName": "automl-dm-1625687301mBS92wfaVVk8-003-1ac43cb1",
        "CandidateStatus": "Completed",
        "CandidateSteps": [
            {
                "CandidateStepArn": "arn:aws:sagemaker:us-east-1:774375974900:processing-job/automl-dm-1625687301-db-1-171ed83849dc417aa1eb46a7a22c91183f227",
                "CandidateStepName": "automl-dm-1625687301-db-1-171ed83849dc417aa1eb46a7a22c91183f227",
                "CandidateStepType": "AWS::SageMaker::ProcessingJob"
            },
            {
                "CandidateStepArn": "arn:aws:sagemaker:us-east-1:774375974900:training-job/automl-dm-1625687301-dpp1-1-e7a81b369ae74776979973e2a1b491831bd",
                "CandidateStepName": "automl-dm-1625687301-dpp1-1-e7a81b369ae74776979973e2a1b491831bd",
                "CandidateStepType": "AWS::SageMaker::TrainingJob"
            },
            {
                "CandidateStepArn": "arn:aws:sagemaker:us-east-1:774375974900:transform-job/automl-dm-1625687301-dpp1-

In [ ]:
print("metric " + str(candidates[0]['FinalAutoMLJobObjectiveMetric']['MetricName']))
for index, candidate in enumerate(candidates):
    print(str(index) + "  " 
        + candidate['CandidateName'] + "  " 
        + str(candidate['FinalAutoMLJobObjectiveMetric']['Value']))

metric validation:accuracy
0  automl-dm-1625687301mBS92wfaVVk8-003-1ac43cb1  0.4006099998950958
1  automl-dm-1625687301mBS92wfaVVk8-002-f4745121  0.5248900055885315
2  automl-dm-1625687301mBS92wfaVVk8-001-bb8b757a  0.5579500198364258


In [ ]:
candidates = automl.list_candidates(job_name=auto_ml_job_name)
if candidates != []:
    best_candidate = automl.best_candidate(
        job_name=auto_ml_job_name 
    )
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))

{
    "CandidateName": "automl-dm-1625687301mBS92wfaVVk8-001-bb8b757a",
    "CandidateProperties": {
        "CandidateArtifactLocations": {
            "Explainability": "s3://sagemaker-us-east-1-774375974900/autopilot/automl-dm-1625687301/documentation/explainability/output"
        }
    },
    "CandidateStatus": "Completed",
    "CandidateSteps": [
        {
            "CandidateStepArn": "arn:aws:sagemaker:us-east-1:774375974900:processing-job/automl-dm-1625687301-db-1-171ed83849dc417aa1eb46a7a22c91183f227",
            "CandidateStepName": "automl-dm-1625687301-db-1-171ed83849dc417aa1eb46a7a22c91183f227",
            "CandidateStepType": "AWS::SageMaker::ProcessingJob"
        },
        {
            "CandidateStepArn": "arn:aws:sagemaker:us-east-1:774375974900:training-job/automl-dm-1625687301-dpp2-1-ef2aaf3aada5437480ecf57071717c0f4a7",
            "CandidateStepName": "automl-dm-1625687301-dpp2-1-ef2aaf3aada5437480ecf57071717c0f4a7",
            "CandidateStepType": "AWS::Sa

In [ ]:
while 'CandidateName' not in best_candidate:
    best_candidate = automl.best_candidate(job_name=auto_ml_job_name)
    print('[INFO] Autopilot Job is generating BestCandidate CandidateName. Please wait. ')
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)
print('[OK] BestCandidate CandidateName generated.')  

[OK] BestCandidate CandidateName generated.


In [ ]:
while 'FinalAutoMLJobObjectiveMetric' not in best_candidate:
    best_candidate = automl.best_candidate(job_name=auto_ml_job_name)
    print('[INFO] Autopilot Job is generating BestCandidate FinalAutoMLJobObjectiveMetric. Please wait. ')
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)
print('[OK] BestCandidate FinalAutoMLJobObjectiveMetric generated.')  

[OK] BestCandidate FinalAutoMLJobObjectiveMetric generated.


In [ ]:
best_candidate_identifier = best_candidate['CandidateName']
print("Candidate name: " + best_candidate_identifier)
print("Metric name: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("Metric value: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

Candidate name: automl-dm-1625687301mBS92wfaVVk8-001-bb8b757a
Metric name: validation:accuracy
Metric value: 0.5579500198364258


In [ ]:
from IPython.core.display import display, HTML
display(
    HTML(
        '<b>Review all <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}?region={}&prefix=autopilot/{}/">output in S3</a></b>'.format(
            bucket, region, auto_ml_job_name
        )
    )
)

In [ ]:
inference_response_keys = ['predicted_label', 'probability']

In [ ]:
autopilot_model = automl.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    candidate=best_candidate,
    inference_response_keys=inference_response_keys,
    predictor_cls=sagemaker.predictor.Predictor,
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer()
)
print('\nEndpoint name:  {}'.format(autopilot_model.endpoint_name))

-----------------!
Endpoint name:  sagemaker-sklearn-automl-2021-07-07-20-33-07-588


In [ ]:
from IPython.core.display import display, HTML
display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST endpoint</a></b>'.format(region, autopilot_model.endpoint_name)))

In [ ]:
sm_runtime = boto3.client('sagemaker-runtime')
review_list = ['This product is great!',
               'OK, but not great.',
               'This is not the right product.']
for review in review_list:
    review = review.replace(",", "")
    response = sm_runtime.invoke_endpoint(
        EndpointName=autopilot_model.endpoint_name,
        ContentType='text/csv', 
        Accept='text/csv', 
        Body=review 
        )
    response_body=response['Body'].read().decode('utf-8').strip().split(',')
    print('Review: ', review, ' Predicated class: {}'.format(response_body[0]))
print("(-1 = Negative, 0=Neutral, 1=Positive)")

Review:  This product is great!  Predicated class: 1
Review:  OK but not great.  Predicated class: 1
Review:  This is not the right product.  Predicated class: -1
(-1 = Negative, 0=Neutral, 1=Positive)
